In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
from tensorflow.keras.models import load_model


In [5]:
#load the trained model,scaler pickle , onehot
# Load trained model
model = load_model('model.h5')

# OneHot Encoder (Geography)
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

# Label Encoder (Gender)
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

# Scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [6]:
#example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [8]:
#one-encode the 'Geography' column
geography_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geography_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\dell\Documents\GitHub\ANN-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
#combine one-hot encoded columns with the rest of the input data
input_data = pd.DataFrame([input_data])
input_data = pd.concat([input_data.drop(columns=['Geography']), geo_encoded_df], axis=1)

In [11]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
#encode categorical features
# Encode Gender
input_data['Gender'] = label_encoder_gender.transform([input_data['Gender']])
input_data

c:\Users\dell\Documents\GitHub\ANN-Classification\venv\Lib\site-packages\sklearn\preprocessing\_label.py:139: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
#scale the input data
input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
#predict the probability of churn
prediction = model.predict(input_data_scaled)
prediction

1/1 [==============================] - 1s 540ms/step


array([[0.02212047]], dtype=float32)

In [15]:
prediction_probability = prediction[0][0]
print(f"Predicted probability of churn: {prediction_probability:.4f}")

Predicted probability of churn: 0.0221


In [16]:
if prediction_probability >= 0.5:
    print("The customer is likely to churn.")   
else:    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
